In [0]:
# inception model
import os
from tensorflow.keras import callbacks

from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                               include_top = False, 
                               weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

#
last_layer = pre_trained_model.get_layer('mixed7') # one of the output layers
last_output = last_layer.output

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation = 'sigmoid')(x)

model = Model(pre_trained_model.input, x)
model.compile(optimizer = RMSprop(lr = 0.0001), 
             loss = 'binary_crossentropy', 
             metrics = ['acc'])

model.summary()

# call backs
class myCallback(callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()


# 
#import tensorflow as tf
#
#model = tf.keras.models.Sequential([
#    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), 
#    tf.keras.layers.MaxPooling2D(2,2), 
#    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
#    tf.keras.layers.MaxPooling2D(2,2), 
#    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
#    tf.keras.layers.MaxPooling2D(2,2), 
#    tf.keras.layers.Flatten(), 
#    tf.keras.layers.Dense(512, activation='relu'), 
#    tf.keras.layers.Dense(1, activation='sigmoid')
#])
#
#
#model.summary()
#
#
#from tensorflow.keras.optimizers import RMSprop
#
#model.compile(loss = 'binary_crossentropy', 
#             optimizer = RMSprop(lr=0.001), 
#             metrics = ['acc'])

# download

!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip
  
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

#
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

#
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

print('total training cat images :', len(os.listdir( train_cats_dir ) ))
print('total training dog images :', len(os.listdir( train_dogs_dir ) ))

print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))

# visualize
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()



#
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255, 
                                  rotation_range=40, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  shear_range=0.2, 
                                  zoom_range=0.2, 
                                  horizontal_flip=True, 
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
  train_dir, 
  target_size=(150, 150), 
  batch_size=20, 
  class_mode='binary') # two classes

#
test_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = test_datagen.flow_from_directory(
  validation_dir, 
  target_size=(150, 150), 
  batch_size = 20, 
  class_mode = 'binary')




# train
histroy = model.fit_generator(
  train_generator, 
  steps_per_epoch = 100, 
  epochs = 15, 
  validation_data = validation_generator, 
  validation_steps = 50, 
    callbacks = [callbacks], 
  verbose = 2)


## run the model
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path='/content/' + fn
  img=image.load_img(path, target_size=(150,150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  image=np.vstack([x])
  
  classes=model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0] > 0:
    print(fn+" is a dog")
  else:
    print(fn + " is a cat")

plt.imshow(img)


## visualize intermediate representation
import numpy as np
import random 
from tensorflow.keras.preprocessing.image import img_to_array, load_img

#
successive_outputs = [layer.output for layer in model.layers[1:]]

visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# take randome cats/dogs
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]

img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size = (150, 150))

x = img_to_array(img)
x = x.reshape((1,) + x.shape)

# rescale
x /= 255.0

successive_feature_maps = visualization_model.predict(x)

layer_names = [layer.name for layer in model.layers]

#
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    
    n_features = feature_map.shape[-1]
    size = feature_map.shape[1]
    
    # tile the images
    display_grid = np.zeros((size, size*n_features))
    
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x+= 128
      x = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i*size:(i+1)*size] = x
      
# display

scale = 20. / n_features
plt.figure(figsize=(scale * n_features, scales))
plt.title (layer_name)
plt.grid(False)
plt.imshow(display_grid, aspect='auto', cmap = 'viridis')


# accuracy and loss of the model

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'acc' ]
val_acc  = history.history[ 'val_acc' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )


# clean up

import os, signal

os.kill(     os.getpid() , 
         signal.SIGKILL
       )

Epoch 1/15
100/100 - 27s - loss: 0.4959 - acc: 0.7660 - val_loss: 0.2695 - val_acc: 0.9180
Epoch 2/15

Reached 95% accuracy so cancelling training!
100/100 - 19s - loss: 0.3778 - acc: 0.8340 - val_loss: 0.2019 - val_acc: 0.9530
